In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import torch
from torch import nn
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from lib.modules import (
    evaluate_loop, 
    read_and_window_session,
    read_session,
    train_loop,
    optimization_loop
)
from lib.utils import (
    plot_and_save_cm,
    summary
)
from lib.models import  MLP, MLP2hl
from tqdm import tqdm
import plotly.express as px
from tabulate import tabulate

In [2]:
raw_dir = Path("/home/mmazeem/datasets/nursingv1")
label_dir = Path("/home/mmazeem/datasets/eating_labels")
WINSIZE = 505
DEVICE = 'cuda'

In [3]:
train_sessions = [25, 67]
test_sessions = [58, 62]

Xs = []
ys = []

for session_idx in train_sessions:
    X,y = read_and_window_session(session_idx, WINSIZE, raw_dir, label_dir)

    Xs.append(X)
    ys.append(y)

Xtr = torch.cat(Xs)
ytr = torch.cat(ys)

Xs = []
ys = []

for session_idx in test_sessions:
    X,y = read_and_window_session(session_idx, WINSIZE, raw_dir, label_dir)

    Xs.append(X)
    ys.append(y)

Xte = torch.cat(Xs)
yte = torch.cat(ys)

In [7]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.p_size = 101
        self.n_channels = 3

        self.lstm = nn.LSTM(
            input_size=self.p_size*self.n_channels,
            hidden_size=64,
            bias=False,
            batch_first=True
        )
        self.mlp = nn.Sequential(
            nn.Linear(in_features=64, out_features=10),
            nn.ReLU(),
            nn.Linear(in_features=10, out_features=1)
        )

    def partition_window(self, x, p_size, n_channels):
        x = Xtr.view(-1, n_channels, WINSIZE)
        x = x.transpose(1,2).view(-1, WINSIZE/p_size, p_size, n_channels)
        x = x.transpose(2,3).flatten(start_dim=2)
        return x

    def forward(self, x):
        x = self.partition_window(x, p_size=self.p_size, n_channels=self.n_channels)
        o, (h,c) = self.lstm(x) # o is shape (batch_size, p_size, 64)
        o = nn.ReLU()(o[:,-1,:])
        logits = self.mlp(o)

        return logits


model = LSTM()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

torch.Size([329500, 1515])